In [37]:
import cv2 as cv
import numpy as np
import glob
from matplotlib import pyplot as plt
import re


#======================= CHESSBOARD PARAMETERS =======================#
chessboard_size = (8, 12)
square_size = 0.019 #m

criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

#======================= CALIB PARAMS =======================#

with open('calib_param.txt', 'r') as f:
    lines = f.readlines()



cleaned = re.sub(r'[\[\]]', '', lines[1])
cleaned = re.sub(r'\s+', ' ', cleaned)
row1 = np.fromstring(cleaned.strip(), sep=' ')

cleaned = re.sub(r'[\[\]]', '', lines[2])
cleaned = re.sub(r'\s+', ' ', cleaned)
row2 = np.fromstring(cleaned.strip(), sep=' ')

cleaned = re.sub(r'[\[\]]', '', lines[3])
cleaned = re.sub(r'\s+', ' ', cleaned)
row3 = np.fromstring(cleaned.strip(), sep=' ')

camera_matrix = np.array([
    row1,
    row2,
    row3
])

cleaned = re.sub(r'[\[\]]', '', lines[5])
cleaned = re.sub(r'\s+', ' ', cleaned)
distortion_coeffs = np.fromstring(cleaned.strip(), sep=' ')

#======================= TCP POSES =======================#

with open('calib_param_hand_eye.txt', 'r') as f:
    lines = f.readlines()

t_gripper2base = []
R_gripper2base = []

for i in range(1, 16):
    pose = (list(map(float, lines[i].strip('[]').strip().rstrip(']').split(','))))
    t = np.array(pose[:3]).reshape(3, 1)      # X Y Z (translation vector)
    rvec = np.array(pose[3:]).reshape(3, 1)  # Rx Ry Rz (rotation vector)

    R, _ = cv.Rodrigues(rvec)  # convert rotation vector to rotation matrix
    t_gripper2base.append(t)
    R_gripper2base.append(R)



#======================= CHESSBOARD COORDINATES =======================#
chessboard_coords = np.zeros((chessboard_size[0] * chessboard_size[1], 3), np.float32)


# Set the X and Y coordinates of the object points (Z remains 0)
chessboard_coords[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2)
chessboard_coords[:, :2] *= square_size


obj_points = []  # 3D points in real world space
img_points = []  # 2D points in image plane

R_target2cam = []
t_target2cam = []

def numerical_sort(value):
    # Extract number from filename, e.g., captured_image12.jpg -> 12
    numbers = re.findall(r'\d+', value)
    return int(numbers[0]) if numbers else -1

calib_images = glob.glob('calibration_images_hand_eye/captured_image*.jpg')
calib_images.sort(key=numerical_sort)

for image in calib_images:
    img = cv.imread(image)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    
    gray = cv.fastNlMeansDenoising(gray)

    cv.imshow('Chessboard', gray)

        
    ret, corners = cv.findChessboardCorners(gray, chessboard_size, cv.CALIB_CB_ADAPTIVE_THRESH + cv.CALIB_CB_NORMALIZE_IMAGE)
        
    if ret:
        
        # Add object points (3D) and image points (2D) after detecting corners
        corners = cv.cornerSubPix(gray, corners, (9,9), (-1,-1), criteria)
        obj_points.append(chessboard_coords)
        img_points.append(corners)

        # Getting rotation and translation vectors of the target
        ret, rvec, tvec = cv.solvePnP(chessboard_coords, corners, camera_matrix, distortion_coeffs)
        if ret:    
            # Draw and display the corners on the chessboard image
            cv.drawChessboardCorners(img, chessboard_size, corners, ret)
            cv.imshow('Chessboard', img)
            print(f"Chessboard corners detected in {image}")
            cv.waitKey(1)

            R, _ = cv.Rodrigues(rvec)
            R_target2cam.append(R)
            t_target2cam.append(tvec)
        else:
            print(f"SolvePnP fail in {image}")
            cv.waitKey(1)
    else:
        print(f"Chessboard corners not detected in {image}")
        cv.waitKey(1)



cv.destroyAllWindows()




Chessboard corners detected in calibration_images_hand_eye\captured_image1.jpg
Chessboard corners detected in calibration_images_hand_eye\captured_image2.jpg
Chessboard corners detected in calibration_images_hand_eye\captured_image3.jpg
Chessboard corners detected in calibration_images_hand_eye\captured_image4.jpg
Chessboard corners detected in calibration_images_hand_eye\captured_image5.jpg
Chessboard corners detected in calibration_images_hand_eye\captured_image6.jpg
Chessboard corners detected in calibration_images_hand_eye\captured_image7.jpg
Chessboard corners detected in calibration_images_hand_eye\captured_image8.jpg
Chessboard corners detected in calibration_images_hand_eye\captured_image9.jpg
Chessboard corners detected in calibration_images_hand_eye\captured_image10.jpg
Chessboard corners detected in calibration_images_hand_eye\captured_image11.jpg
Chessboard corners detected in calibration_images_hand_eye\captured_image12.jpg
Chessboard corners detected in calibration_images

In [38]:

R_cam2gripper, t_cam2gripper = cv.calibrateHandEye(
    R_gripper2base, t_gripper2base,  # robot
    R_target2cam, t_target2cam,      # camera
    method=cv.CALIB_HAND_EYE_TSAI
)

print(R_cam2gripper, t_cam2gripper)

with open('calib_param_hand_eye.txt', 'a') as file:
    file.write("\n=============== R_cam2gripper ===============\n")
    file.write(str(R_cam2gripper) + '\n')
    file.write("=============== t_cam2gripper ===============\n")
    file.write(str(t_cam2gripper) + '\n')

[[ 0.00251655  0.999976    0.00645561]
 [-0.99995384  0.00257625 -0.00925623]
 [-0.00927264 -0.00643202  0.99993632]] [[-0.05920206]
 [-0.01402861]
 [-0.11490674]]


In [36]:
print(distortion_coeffs)
print(pose)


[ 0.10205353 -0.22224811 -0.00141093  0.00122758  0.05709275]
[-0.17236, 0.79189, 0.16915, 0.20328, 2.8953, -0.08701]
